# Install dependencies


In [1]:
!pip install -qU torch==2.7.1 torchvision==0.22.1 "ray[client,train]"==2.47.1

# Import dependencies


In [2]:
import os
from typing import Dict

import torch
from filelock import FileLock
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import Normalize, ToTensor

import ray
import ray.train
from ray.train import ScalingConfig
from ray.train.torch import TorchTrainer

# Define functions


In [3]:
def get_dataloaders(batch_size):
    # Transform to normalize the input images
    transform = transforms.Compose([ToTensor(), Normalize((0.5,), (0.5,))])

    with FileLock(os.path.expanduser("~/data.lock")):
        # Download training data from open datasets
        training_data = datasets.FashionMNIST(
            root="~/data",
            train=True,
            download=True,
            transform=transform,
        )

        # Download test data from open datasets
        test_data = datasets.FashionMNIST(
            root="~/data",
            train=False,
            download=True,
            transform=transform,
        )

    # Create data loaders
    train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_data, batch_size=batch_size)

    return train_dataloader, test_dataloader


# Model Definition
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(512, 10),
            nn.ReLU(),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


def train_func_per_worker(config: Dict):
    lr = config["lr"]
    epochs = config["epochs"]
    batch_size = config["batch_size_per_worker"]

    # Get dataloaders inside the worker training function
    train_dataloader, test_dataloader = get_dataloaders(batch_size=batch_size)

    # [1] Prepare Dataloader for distributed training
    # Shard the datasets among workers and move batches to the correct device
    # =======================================================================
    train_dataloader = ray.train.torch.prepare_data_loader(train_dataloader)
    test_dataloader = ray.train.torch.prepare_data_loader(test_dataloader)

    model = NeuralNetwork()

    # [2] Prepare and wrap your model with DistributedDataParallel
    # Move the model to the correct GPU/CPU device
    # ============================================================
    model = ray.train.torch.prepare_model(model)

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)

    # Model training loop
    for epoch in range(epochs):
        if ray.train.get_context().get_world_size() > 1:
            # Required for the distributed sampler to shuffle properly across epochs.
            train_dataloader.sampler.set_epoch(epoch)

        model.train()
        print(f"Train Epoch {epoch}")
        for X, y in train_dataloader:
            pred = model(X)
            loss = loss_fn(pred, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        model.eval()
        test_loss, num_correct, num_total = 0, 0, 0
        with torch.no_grad():
            print(f"Test Epoch {epoch}")
            for X, y in test_dataloader:
                pred = model(X)
                loss = loss_fn(pred, y)

                test_loss += loss.item()
                num_total += y.shape[0]
                num_correct += (pred.argmax(1) == y).sum().item()

        test_loss /= len(test_dataloader)
        accuracy = num_correct / num_total

        # [3] Report metrics to Ray Train
        # ===============================
        ray.train.report(metrics={"loss": test_loss, "accuracy": accuracy})


@ray.remote
def train_fashion_mnist(num_workers=2, use_gpu=False):
    global_batch_size = 32

    train_config = {
        "lr": 1e-3,
        "epochs": 10,
        "batch_size_per_worker": global_batch_size // num_workers,
    }

    # Configure computation resources
    scaling_config = ScalingConfig(num_workers=num_workers, use_gpu=use_gpu)

    # Initialize a Ray TorchTrainer
    trainer = TorchTrainer(
        train_loop_per_worker=train_func_per_worker,
        train_loop_config=train_config,
        scaling_config=scaling_config,
    )

    # [4] Start distributed training
    # Run `train_func_per_worker` on all workers
    # =============================================
    result = trainer.fit()
    print(f"Training result: {result}")

# Connect to Ray Cluster


In [4]:
ray.init(
    address="ray://raycluster-gvt-head-svc.default.svc.cluster.local:10001",
    runtime_env={"pip": ["torch==2.7.1", "torchvision==0.22.1"]},
)

2025-06-24 09:19:28,372	INFO client_builder.py:242 -- Passing the following kwargs to ray.init() on the server: log_to_driver
2025-06-24 09:19:57,804	WARNING utils.py:1565 -- Python patch version mismatch: The cluster was started with:
    Ray: 2.47.1
    Python: 3.9.23
This process on Ray Client was started with:
    Ray: 2.47.1
    Python: 3.9.13



Python version:,3.9.23
Ray version:,2.47.1
Dashboard:,http://10.0.2.175:8265


# Train model remotely


In [5]:
ray.get(train_fashion_mnist.remote(num_workers=7, use_gpu=False))

(autoscaler +7s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
(autoscaler +7s) Adding 1 node(s) of type cpu-group.
(autoscaler +7s) Resized to 7 CPUs.
(train_fashion_mnist pid=326, ip=10.0.3.73) 
(train_fashion_mnist pid=326, ip=10.0.3.73) View detailed results here: /home/ray/ray_results/TorchTrainer_2025-06-24_02-22-04
(train_fashion_mnist pid=326, ip=10.0.3.73) To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2025-06-24_01-30-39_079090_1/artifacts/2025-06-24_02-22-04/TorchTrainer_2025-06-24_02-22-04/driver_artifacts`
(autoscaler +2m9s) Adding 1 node(s) of type cpu-group.
(autoscaler +2m9s) Resized to 14 CPUs.
(autoscaler +2m9s) No available node types can fulfill resource requests {'bundle_group_2e3877008cb7a6f37aaeebd5829801000000': 0.001}*1. Add suitable node types to this cluster to resolve this issue.
(autoscaler +2m14s) No available node types can fulfill resource requests {'

(RayTrainWorker pid=405, ip=10.0.3.215) Setting up process group for: env:// [rank=0, world_size=7]
(TorchTrainer pid=326, ip=10.0.3.215) Started distributed worker processes: 
(TorchTrainer pid=326, ip=10.0.3.215) - (node_id=2cae2784a256b462b779df5d8d4b137e0d697a40b20ec7210b47f258, ip=10.0.3.215, pid=405) world_rank=0, local_rank=0, node_rank=0
(TorchTrainer pid=326, ip=10.0.3.215) - (node_id=2cae2784a256b462b779df5d8d4b137e0d697a40b20ec7210b47f258, ip=10.0.3.215, pid=404) world_rank=1, local_rank=1, node_rank=0
(TorchTrainer pid=326, ip=10.0.3.215) - (node_id=2cae2784a256b462b779df5d8d4b137e0d697a40b20ec7210b47f258, ip=10.0.3.215, pid=407) world_rank=2, local_rank=2, node_rank=0
(TorchTrainer pid=326, ip=10.0.3.215) - (node_id=2cae2784a256b462b779df5d8d4b137e0d697a40b20ec7210b47f258, ip=10.0.3.215, pid=409) world_rank=3, local_rank=3, node_rank=0
(TorchTrainer pid=326, ip=10.0.3.215) - (node_id=2cae2784a256b462b779df5d8d4b137e0d697a40b20ec7210b47f258, ip=10.0.3.215, pid=408) world_ra

(RayTrainWorker pid=405, ip=10.0.3.215) Train Epoch 0
(RayTrainWorker pid=404, ip=10.0.3.215) Train Epoch 0
(RayTrainWorker pid=407, ip=10.0.3.215) Train Epoch 0
(RayTrainWorker pid=409, ip=10.0.3.215) Train Epoch 0
(RayTrainWorker pid=408, ip=10.0.3.215) Train Epoch 0
(RayTrainWorker pid=406, ip=10.0.3.215) Train Epoch 0
(RayTrainWorker pid=650, ip=10.0.3.73) Train Epoch 0
(RayTrainWorker pid=405, ip=10.0.3.215) Test Epoch 0
(RayTrainWorker pid=404, ip=10.0.3.215) Test Epoch 0
(RayTrainWorker pid=407, ip=10.0.3.215) Test Epoch 0
(RayTrainWorker pid=409, ip=10.0.3.215) Test Epoch 0
(RayTrainWorker pid=408, ip=10.0.3.215) Test Epoch 0
(RayTrainWorker pid=406, ip=10.0.3.215) Test Epoch 0
(RayTrainWorker pid=650, ip=10.0.3.73) Test Epoch 0
(RayTrainWorker pid=405, ip=10.0.3.215) Train Epoch 1
(RayTrainWorker pid=404, ip=10.0.3.215) Train Epoch 1
(RayTrainWorker pid=407, ip=10.0.3.215) Train Epoch 1
(train_fashion_mnist pid=326, ip=10.0.3.73) 
(train_fashion_mnist pid=326, ip=10.0.3.73) Tr

# Shut down Ray workers


In [6]:
ray.shutdown()

(train_fashion_mnist pid=326, ip=10.0.3.73) 
(train_fashion_mnist pid=326, ip=10.0.3.73) Training completed after 10 iterations at 2025-06-24 02:27:02. Total running time: 4min 57s
(train_fashion_mnist pid=326, ip=10.0.3.73) 
(train_fashion_mnist pid=326, ip=10.0.3.73) Training result: Result(
(train_fashion_mnist pid=326, ip=10.0.3.73)   metrics={'loss': 0.3560600821211721, 'accuracy': 0.8761371588523443},
(train_fashion_mnist pid=326, ip=10.0.3.73)   path='/home/ray/ray_results/TorchTrainer_2025-06-24_02-22-04/TorchTrainer_b1496_00000_0_2025-06-24_02-22-05',
(train_fashion_mnist pid=326, ip=10.0.3.73)   filesystem='local',
(train_fashion_mnist pid=326, ip=10.0.3.73)   checkpoint=None
(train_fashion_mnist pid=326, ip=10.0.3.73) )


(train_fashion_mnist pid=326, ip=10.0.3.73) Wrote the latest version of all result files and experiment state to '/home/ray/ray_results/TorchTrainer_2025-06-24_02-22-04' in 0.0047s.
